# Joe Stanley
### ECE523 - EXAM1

In [1]:
# Import Necessary Libraries
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate as tab
import electricpy as ep
from electricpy.constants import *

![P-1-1.png](P-1-1.png)

![P-1-2.png](P-1-2.png)

### Solution:

***a)***
For loads described only by their positive sequence impedance, it is common to assume the negative sequence impedance as equal (and opposite where applicable in transformer phase shifts) to the positive sequence impedance, and it is also common to assume the zero sequence impedance as three times the positive sequence. That is to say: $$Z_2 \approx Z_1 \qquad Z_0 \approx 3\cdot Z_1$$ For fault analysis, short-circuit MVA or short-circuit current are also valuable pieces of information. Power flow information could be calculated at a steady-state value for unfaulted conditions using the positive sequence impedance to evaluate the power-flow results and treating the "load" as an infinite bus whose voltage magnitude and angle was fixed.

***b)***
Treating the load as constant impedance load would ultimately remove the fixation of voltage magnitude and angle for the "load", however, it would largely leave the impedance relationships unchanged.

***c)***
Treating the load as a constant current load may allow the assumption that the load's impedance will not affect the fault analysis, and that the fault studies may be conducted with impedance matrices that do not include the "load". The "load" current may not be neglected, however. Instead, the load current (being fixed) would be added to the fault current and any other relational power flow currents in much the same way that power-flow currents are treated in textbook-fault studies.

***d)***
To fully account for a fixed power (or constant power) load, the solution method of solution should incorporate sequence component fault analysis prior to a power-flow calculation that includes the fictitious fault bus, accounting for the power consumption for the fault. This will allow for proper accounting of both the fault currents and the constant power currents, which will (inevitably) be affected by the fault's impact on bus voltages.

***e)***
To account for a load that incorporates 40% constant power consumption, 40% constant current consumption, and 20% constant impedance would require decomposing the single load into three discrete "component" loads that represent each type of load characteristic. This would allow the composition of the various methods previously described to effectively represent the system.

![P-2-1.png](P-2-1.png)

![P-2-2.png](P-2-2.png)

### Solution:

***I.***

| Description:      | $1\phi$   $\text{MVA}_{sc}$ | $3\phi$   $\text{MVA}_{sc}$ |
|-------------------|-----------------------------|-----------------------------|
| All On Line       | {{a_1ph}}                   | {{a_3ph}}                   |
| Line 1 Fail       | {{b1_1ph}}                  | {{b1_3ph}}                  |
| Line 2 Fail       | {{b2_1ph}}                  | {{b2_3ph}}                  |
| Line 3 Fail       | {{b3_1ph}}                  | {{b3_3ph}}                  |
| Line 4 Fail       | {{b4_1ph}}                  | {{b4_3ph}}                  |
| Source 2 Off Line | {{c1_1ph}}                  | {{c1_3ph}}                  |
| Source 3 Off Line | {{c2_1ph}}                  | {{c2_3ph}}                  |


***II.***
The results will change if the point of common coupling was refferred to the low-voltage side of the transformer. This is beacuse the system would then have to be treated as a (minimum) 4 bus network and thus include the transformer impedance and phase shift in the thevenin impedance calculations.

#### Numerical Analysis and Solution:

In [5]:
# Define Per-Unit Terms
Sbase = 100*M
VbaseT = 230*k
VbaseG = 24*k
ZbaseT = ep.zpu(Sbase,VLL=VbaseT)
ZbaseG = ep.zpu(Sbase,VLL=VbaseG)
IbaseT = ep.ipu(Sbase,VLL=VbaseT)
IbaseG = ep.ipu(Sbase,VLL=VbaseG)

# Define Sequence Impedances
ZS11 = 0.1J
ZS10 = 0.1J
XFM1 = 0.2J
XFM0 = 0.2J
ZS21 = 0.2J
ZS20 = 0.35J
ZS31 = 0.2J
ZS30 = 0.7J
ZL11 = 0.01+0.15J
ZL10 = 0.06+0.5J
ZL21 = 0.05+0.5J
ZL20 = 0.20+2.0J
ZL31 = 0.02+0.2J
ZL30 = 0.07+0.7J
ZL41 = 0.02+0.2J
ZL40 = 0.07+0.7J    

# Define Function to Provide Impedance Matrices
def zmatrix(ZL1=True,ZL2=True,ZL3=True,ZL4=True,S2=True,S3=True,verbose=False):
    # Define Pos/Neg Seq Y-Bus Matrix
    y12 = np.array([
        [1/(ZS11+XFM1)+int(ZL1)/ZL11+int(ZL2)/ZL21, -int(ZL1)/ZL11, -int(ZL2)/ZL21],
        [-int(ZL1)/ZL11, int(ZL1)/ZL11+1/(int(ZL3)*ZL31+int(ZL4)*ZL41)+int(S2)/ZS21, -1/(int(ZL3)*ZL31+int(ZL4)*ZL41)],
        [-int(ZL2)/ZL21, -1/(int(ZL3)*ZL31+int(ZL4)*ZL41), int(ZL2)/ZL21+1/(int(ZL3)*ZL31+int(ZL4)*ZL41)+int(S3)/ZS31],
    ])
    # Define Zero Seq Y-Bus Matrix
    y0 = np.array([
        [1/(ZS10+XFM0)+int(ZL1)/ZL10+int(ZL2)/ZL20, -int(ZL1)/ZL10, -int(ZL2)/ZL20],
        [-int(ZL1)/ZL10, int(ZL1)/ZL10+1/(int(ZL3)*ZL30+int(ZL4)*ZL40)+int(S2)/ZS20, -1/(int(ZL3)*ZL30+int(ZL4)*ZL40)],
        [-int(ZL2)/ZL20, -1/(int(ZL3)*ZL30+int(ZL4)*ZL40), int(ZL2)/ZL20+1/(int(ZL3)*ZL30+int(ZL4)*ZL40)+int(S3)/ZS30],
    ])
    # Calculate Z-Bus Matrices
    zbus1 = np.linalg.inv(y12)
    zbus2 = np.linalg.inv(y12)
    zbus0 = np.linalg.inv(y0)
    # Print if Necessary
    if verbose:
        print("\nPositive Sequence Z-Bus:\n",tab(np.asarray(np.around(zbus1,3),dtype=str),tablefmt="fancy_grid"),sep='')
        print("\nNegative Sequence Z-Bus:\n",tab(np.asarray(np.around(zbus2,3),dtype=str),tablefmt="fancy_grid"),sep='')
        print("\nZero Sequence Z-Bus:\n",tab(np.asarray(np.around(zbus0,3),dtype=str),tablefmt="fancy_grid"),sep='')
    # Return Results
    return(zbus1,zbus2,zbus0)

# Define BUSnn Formulator
def BUSn(tup,ind=0):
    z1, z2, z0 = tup
    z1n = z1[ind][ind]
    z2n = z2[ind][ind]
    z0n = z0[ind][ind]
    return([z0n,z1n,z2n])

# Demonstrate Validity
print("Z-Bus, All On-Line")
z1,z2,z0 = zmatrix(verbose=True)

# A) Evaluate MVAsc for All On-Line
a_3ph = round(ep.fault.phs3mvasc(1.0,BUSn(zmatrix()),Sbase=Sbase),2)
a_1ph = round(ep.fault.phs1mvasc(1.0,BUSn(zmatrix()),Sbase=Sbase),2)

# B) Evaluate MVAsc for Each One Line Failure
b1_3ph = round(ep.fault.phs3mvasc(1.0,BUSn(zmatrix(ZL1=False)),Sbase=Sbase),2)
b1_1ph = round(ep.fault.phs1mvasc(1.0,BUSn(zmatrix(ZL1=False)),Sbase=Sbase),2)
b2_3ph = round(ep.fault.phs3mvasc(1.0,BUSn(zmatrix(ZL2=False)),Sbase=Sbase),2)
b2_1ph = round(ep.fault.phs1mvasc(1.0,BUSn(zmatrix(ZL2=False)),Sbase=Sbase),2)
b3_3ph = round(ep.fault.phs3mvasc(1.0,BUSn(zmatrix(ZL3=False)),Sbase=Sbase),2)
b3_1ph = round(ep.fault.phs1mvasc(1.0,BUSn(zmatrix(ZL3=False)),Sbase=Sbase),2)
b4_3ph = round(ep.fault.phs3mvasc(1.0,BUSn(zmatrix(ZL4=False)),Sbase=Sbase),2)
b4_1ph = round(ep.fault.phs1mvasc(1.0,BUSn(zmatrix(ZL4=False)),Sbase=Sbase),2)

# C) Evaluate MVAsc for Each Source Off-Line
c1_3ph = round(ep.fault.phs3mvasc(1.0,BUSn(zmatrix(S2=False)),Sbase=Sbase),2)
c1_1ph = round(ep.fault.phs1mvasc(1.0,BUSn(zmatrix(S2=False)),Sbase=Sbase),2)
c2_3ph = round(ep.fault.phs3mvasc(1.0,BUSn(zmatrix(S3=False)),Sbase=Sbase),2)
c2_1ph = round(ep.fault.phs1mvasc(1.0,BUSn(zmatrix(S3=False)),Sbase=Sbase),2)

Z-Bus, All On-Line

Positive Sequence Z-Bus:
╒═════════════════╤═════════════════╤═════════════════╕
│ (0.003+0.129j)  │ (-0+0.069j)     │ (-0.002+0.045j) │
├─────────────────┼─────────────────┼─────────────────┤
│ (-0+0.069j)     │ (0.002+0.111j)  │ (-0.002+0.044j) │
├─────────────────┼─────────────────┼─────────────────┤
│ (-0.002+0.045j) │ (-0.002+0.044j) │ (0.003+0.126j)  │
╘═════════════════╧═════════════════╧═════════════════╛

Negative Sequence Z-Bus:
╒═════════════════╤═════════════════╤═════════════════╕
│ (0.003+0.129j)  │ (-0+0.069j)     │ (-0.002+0.045j) │
├─────────────────┼─────────────────┼─────────────────┤
│ (-0+0.069j)     │ (0.002+0.111j)  │ (-0.002+0.044j) │
├─────────────────┼─────────────────┼─────────────────┤
│ (-0.002+0.045j) │ (-0.002+0.044j) │ (0.003+0.126j)  │
╘═════════════════╧═════════════════╧═════════════════╛

Zero Sequence Z-Bus:
╒═════════════════╤═════════════════╤═════════════════╕
│ (0.005+0.205j)  │ (-0.004+0.081j) │ (-0.003+0.06j)  │
├──────────

![P-3.png](P-3.png)

### Solution:

*ASSUMPTIONS:*

- The relays (R1/R2) are connected to a discrete section of line as close as possible to the bus, but not on it. Therefore, the assumption is made that if a fault occurs on the bus, that bus's relay sees the fault as behind itself.
- Voltage from the sources is regulated at the terminals, not at the internal voltage point.
- The Relays are "looking into" the line

***A.*** All calculations performed in following code section (Numerical Analysis and Solution).

| Location       | Single-Line-to-Ground (Per-Unit) | Line-to-Line (Per-Unit)  | Double-Line-to-Ground (Per-Unit) |
|----------------|----------------------------------|--------------------------|----------------------------------|
| Bus 1          | V:{{v1_slg}}, I:{{i1_slg}}       | V:{{v1_ll}}, I:{{i1_ll}} | V:{{v1_dlg}}, I:{{i1_dlg}}       |
| 33% From Bus 1 | V:{{v2_slg}}, I:{{i2_slg}}       | V:{{v2_ll}}, I:{{i2_ll}} | V:{{v2_dlg}}, I:{{i2_dlg}}       |
| 67% From Bus 1 | V:{{v3_slg}}, I:{{i3_slg}}       | V:{{v3_ll}}, I:{{i3_ll}} | V:{{v3_dlg}}, I:{{i3_dlg}}       |
| Bus 2          | V:{{v4_slg}}, I:{{i4_slg}}       | V:{{v4_ll}}, I:{{i4_ll}} | V:{{v4_dlg}}, I:{{i4_dlg}}       |

***B.***

![Single-Line-to-Ground.png](Single-Line-to-Ground.png)

![Line-to-Line.png](Line-to-Line.png)

![Double-Line-to-Ground.png](Double-Line-to-Ground.png)


***C.*** All calculations performed in following code section (Numerical Analysis and Solution).

![Single-Line-to-Ground%20Angle%20Differences.png](Single-Line-to-Ground%20Angle%20Differences.png)

![Line-to-Line%20Angle%20Differences.png](Line-to-Line%20Angle%20Differences.png)

![Double-Line-to-Ground%20Angle%20Differences.png](Double-Line-to-Ground%20Angle%20Differences.png)

***D.***
These results exemplify what one would expect for the various types of faults on a simple single line system. It is encouraging to see the relationships between the voltage magnitudes as shown in section B, and it is also interesting to see the "locked" variance between angles as shown in section C. It is interesting to see that these angle relations seem to remain largely unchanged over the entire length of the line. This implies that the angle can be used as a factor of protection logic as it is not affected by the fault location. Then, of course, the supervisory control decision may be made by magnitude detection.


#### Numerical Analysis and Solution:

In [4]:
# Define Per-Unit Bases
Sbase = 100*M
VbaseT = 230*k
VbaseS = (VbaseT*13.2/220)*k
VbaseR = 24*k
ZbaseT = ep.zpu(Sbase,VLL=VbaseT)
ZbaseS = ep.zpu(Sbase,VLL=VbaseS)
ZbaseR = ep.zpu(Sbase,VLL=VbaseR)

# Define System Parameters
ZL1 = ep.phasor(0.8,85)
ZL0 = ep.phasor(3,80)
ZT1 = ep.puchgbase(ep.rxrecompose(0.1,15),ep.zpu(150*M,220*k),ZbaseT)
ZT2 = ep.puchgbase(ep.rxrecompose(0.1,15),ep.zpu(75*M,230*k),ZbaseT)
ZS1 = ep.puchgbase(0.3j,ep.zpu(150*M,13.8*k),ZbaseS)
ZS0 = ep.puchgbase(8.0+0.01j,ep.zpu(150*M,13.8*k),ZbaseS)
ZR1 = ep.puchgbase(0.3j,ep.zpu(75*M,24*k),ZbaseR)
ZR0 = ep.puchgbase(8.0+0.01j,ep.zpu(75*M,24*k),ZbaseR)

# A.1)
# General Terms
Vth = 1
Zth = np.array([ZL0,ZL1+ZT2,ZL1+ZT2]) # Sequence Impedances
Zbus = np.array([ZL0,ZL1+ZT2,ZL1+ZT2])
# Single Line to Ground
# Faulted Terms
i = ep.fault.phs1g(Vth,Zth)
v = ep.abc_to_seq([Vth,Vth*ep.phs(-120),Vth*ep.phs(-240)]) - Zth * i
# Printable Terms
i1_slg = ep.clatex(-i)
v1_slg = ep.clatex(v)
# Line to Line
# Faulted Terms
i = ep.fault.phs2(Vth,Zth)
v = ep.abc_to_seq([Vth,Vth*ep.phs(-120),Vth*ep.phs(-240)]) - Zth * i
# Printable Terms
i1_ll = ep.clatex(-i)
v1_ll = ep.clatex(v)
# Double Line to Ground
# Faulted Terms
i = ep.fault.phs2g(Vth,Zth)
v = ep.abc_to_seq([Vth,Vth*ep.phs(-120),Vth*ep.phs(-240)]) - Zth * i
# Printable Terms
i1_dlg = ep.clatex(-i)
v1_dlg = ep.clatex(v)

# A.2)
# General Terms
Vth = 1
Zbus = np.array([0,ZT1,ZT1])
z0 = 0.33*ZL0
z1 = (0.33*ZL1+ZT1)
z2 = z1
Zth = np.array([z0,z1,z2]) # Sequence Impedances
# Single Line to Ground
# Faulted Terms
i = ep.fault.phs1g(Vth,Zth)
v = ep.abc_to_seq([Vth,Vth*ep.phs(-120),Vth*ep.phs(-240)]) - Zbus * i
# Printable Terms
i2_slg = ep.clatex(i)
v2_slg = ep.clatex(v)
# Line to Line
# Faulted Terms
i = ep.fault.phs2(Vth,Zth)
v = ep.abc_to_seq([Vth,Vth*ep.phs(-120),Vth*ep.phs(-240)]) - Zbus * i
# Printable Terms
i2_ll = ep.clatex(i)
v2_ll = ep.clatex(v)
# Double Line to Ground
# Faulted Terms
i = ep.fault.phs2g(Vth,Zth)
v = ep.abc_to_seq([Vth,Vth*ep.phs(-120),Vth*ep.phs(-240)]) - Zbus * i
# Printable Terms
i2_dlg = ep.clatex(i)
v2_dlg = ep.clatex(v)

# A.3)
# General Terms
z0 = 0.67*ZL0
z1 = (0.67*ZL1+ZT1)
z2 = z1
Zth = np.array([z0,z1,z2]) # Sequence Impedances
# Single Line to Ground
# Faulted Terms
i = ep.fault.phs1g(Vth,Zth)
v = ep.abc_to_seq([Vth,Vth*ep.phs(-120),Vth*ep.phs(-240)]) - Zbus * i
# Printable Terms
i3_slg = ep.clatex(i)
v3_slg = ep.clatex(v)
# Line to Line
# Faulted Terms
i = ep.fault.phs2(Vth,Zth)
v = ep.abc_to_seq([Vth,Vth*ep.phs(-120),Vth*ep.phs(-240)]) - Zbus * i
# Printable Terms
i3_ll = ep.clatex(i)
v3_ll = ep.clatex(v)
# Double Line to Ground
# Faulted Terms
i = ep.fault.phs2g(Vth,Zth)
v = ep.abc_to_seq([Vth,Vth*ep.phs(-120),Vth*ep.phs(-240)]) - Zbus * i
# Printable Terms
i3_dlg = ep.clatex(i)
v3_dlg = ep.clatex(v)

# A.4)
# General Terms
z0 = ZL0
z1 = ZL1+ZT1
z2 = z1
Zth = np.array([z0,z1,z2]) # Sequence Impedances
# Single Line to Ground
# Faulted Terms
i = ep.fault.phs1g(Vth,Zth)
v = ep.abc_to_seq([Vth,Vth*ep.phs(-120),Vth*ep.phs(-240)]) - Zbus * i
# Printable Terms
i4_slg = ep.clatex(i)
v4_slg = ep.clatex(v)
# Line to Line
# Faulted Terms
i = ep.fault.phs2(Vth,Zth)
v = ep.abc_to_seq([Vth,Vth*ep.phs(-120),Vth*ep.phs(-240)]) - Zbus * i
# Printable Terms
i4_ll = ep.clatex(i)
v4_ll = ep.clatex(v)
# Double Line to Ground
# Faulted Terms
i = ep.fault.phs2g(Vth,Zth)
v = ep.abc_to_seq([Vth,Vth*ep.phs(-120),Vth*ep.phs(-240)]) - Zbus * i
# Printable Terms
i4_dlg = ep.clatex(i)
v4_dlg = ep.clatex(v)

# B)
LineLen = np.arange(1,101,1)
# Define Simple Plotter Function
def VoverLinePlot(faultfunc,title,plot=True):
    # Define Initial Arrays
    V0 = np.array([])
    V1 = np.array([])
    V2 = np.array([])
    Va = np.array([])
    wvi1 = np.array([])
    wvi2 = np.array([])
    wvi0 = np.array([])
    wi20 = np.array([])
    for L in LineLen:
        z0 = (L/100)*ZL0
        z1 = (L/100)*ZL1+ZT1
        z2 = z1
        Zth = np.array([z0,z1,z2]) # Sequence Impedances
        i = faultfunc(Vth,Zth)
        v = ep.abc_to_seq([Vth,Vth*ep.phs(-120),Vth*ep.phs(-240)]) - Zbus * i
        #ep.cprint(ep.seq_to_phs(v))
        va,vb,vc = np.abs(ep.seq_to_phs(v))
        # Calculate Voltage Magnitudes
        Va = np.append(Va,va)
        v0, v1, v2 = np.abs(v)
        V0 = np.append(V0,v0)
        V1 = np.append(V1,v1)
        V2 = np.append(V2,v2)
        # Calculate Angle Differences
        wvi1 = np.append(wvi1,np.angle(v1,True)-np.angle(i[1],True))
        wvi2 = np.append(wvi2,np.angle(v2,True)-np.angle(i[2],True))
        wvi0 = np.append(wvi0,np.angle(v0,True)-np.angle(i[0],True))
        wi20 = np.append(wi20,np.angle(i[2],True)-np.angle(i[0],True))
    # Plot Voltage Magnitudes
    plt.figure()
    plt.plot(LineLen,V0,label='Zero-Sequence')
    plt.plot(LineLen,V1,label='Pos-Sequence')
    plt.plot(LineLen,V2,label='Neg-Sequence')
    plt.plot(LineLen,Va,label="Va")
    plt.title(title)
    plt.xlabel("Percent of Line Length")
    plt.ylabel("Per-Unit Voltage")
    plt.legend()
    plt.savefig(title+".png")
    if plot==True:
        plt.show()
    else:
        plt.close()
    # Plot Angle Differences
    plt.figure()
    plt.plot(LineLen,wvi1,label="$\\theta_{v1}-\\theta_{i1}$",linestyle='-')
    plt.plot(LineLen,wvi2,label="$\\theta_{v2}-\\theta_{i2}$",linestyle='--')
    plt.plot(LineLen,wvi0,label="$\\theta_{v0}-\\theta_{i0}$",linestyle='-.')
    plt.plot(LineLen,wi20,label="$\\theta_{i2}-\\theta_{i0}$",linestyle=':')
    plt.title(title+" Angle Differences")
    plt.xlabel("Percent of Line Length")
    plt.ylabel("Angle-Difference (Degrees)")
    plt.legend()
    plt.savefig(title+" Angle Differences.png")
    if plot==True:
        plt.show()
    else:
        plt.close()

# Generate Plots for Each Fault Type
VoverLinePlot(ep.fault.phs1g,"Single-Line-to-Ground",False)
VoverLinePlot(ep.fault.phs2g,"Double-Line-to-Ground",plot=False)
VoverLinePlot(ep.fault.phs2,"Line-to-Line",plot=False)